In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import pandas as pd
from env import github_token, github_username
import wrangle
#import prepare
import acquire
#import explore
#import model

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filter="ignore"

from bs4 import BeautifulSoup
from mergedeep import merge

In [2]:
#Acquire the URLS for the Repositories to explore

headers = acquire.headers
endpoint = "https://api.github.com/search/repositories"
query = "musicbot"
sort = "stars"
per_page = 150
order = "desc"
page = 1

##Make the request
response = requests.get(f"{endpoint}?q={query}&sort={sort}&per_page={per_page}&order={order}&page={page}", headers=headers)
payload = response.json()
payload['items']

[{'id': 65681319,
  'node_id': 'MDEwOlJlcG9zaXRvcnk2NTY4MTMxOQ==',
  'name': 'MusicBot',
  'full_name': 'jagrosh/MusicBot',
  'private': False,
  'owner': {'login': 'jagrosh',
   'id': 12754382,
   'node_id': 'MDQ6VXNlcjEyNzU0Mzgy',
   'avatar_url': 'https://avatars.githubusercontent.com/u/12754382?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/jagrosh',
   'html_url': 'https://github.com/jagrosh',
   'followers_url': 'https://api.github.com/users/jagrosh/followers',
   'following_url': 'https://api.github.com/users/jagrosh/following{/other_user}',
   'gists_url': 'https://api.github.com/users/jagrosh/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/jagrosh/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/jagrosh/subscriptions',
   'organizations_url': 'https://api.github.com/users/jagrosh/orgs',
   'repos_url': 'https://api.github.com/users/jagrosh/repos',
   'events_url': 'https://api.github.com/users/jagrosh/events{

In [3]:
page = 2

##Make the request
response = requests.get(f"{endpoint}?q={query}&sort={sort}&per_page={per_page}&order={order}&page={page}", headers=headers)
payload2 = response.json()
payload2['items']

[{'id': 94307500,
  'node_id': 'MDEwOlJlcG9zaXRvcnk5NDMwNzUwMA==',
  'name': 'Kiu',
  'full_name': 'BjoernPetersen/Kiu',
  'private': False,
  'owner': {'login': 'BjoernPetersen',
   'id': 2513555,
   'node_id': 'MDQ6VXNlcjI1MTM1NTU=',
   'avatar_url': 'https://avatars.githubusercontent.com/u/2513555?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/BjoernPetersen',
   'html_url': 'https://github.com/BjoernPetersen',
   'followers_url': 'https://api.github.com/users/BjoernPetersen/followers',
   'following_url': 'https://api.github.com/users/BjoernPetersen/following{/other_user}',
   'gists_url': 'https://api.github.com/users/BjoernPetersen/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/BjoernPetersen/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/BjoernPetersen/subscriptions',
   'organizations_url': 'https://api.github.com/users/BjoernPetersen/orgs',
   'repos_url': 'https://api.github.com/users/BjoernPetersen/repos

In [4]:
#Convert the reponse to a data frame
df1 = pd.DataFrame(payload['items'])

In [5]:
#Convert the reponse to a data frame
df2 = pd.DataFrame(payload2['items'])

In [6]:
df1

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,is_template,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,permissions,score
0,65681319,MDEwOlJlcG9zaXRvcnk2NTY4MTMxOQ==,MusicBot,jagrosh/MusicBot,False,"{'login': 'jagrosh', 'id': 12754382, 'node_id'...",https://github.com/jagrosh/MusicBot,🎶 A Discord music bot that's easy to set up an...,False,https://api.github.com/repos/jagrosh/MusicBot,...,False,False,"[discord, discord-bot, java, jda, music, music...",public,1732,135,3281,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
1,48729353,MDEwOlJlcG9zaXRvcnk0ODcyOTM1Mw==,MusicBot,Just-Some-Bots/MusicBot,False,"{'login': 'Just-Some-Bots', 'id': 22779351, 'n...",https://github.com/Just-Some-Bots/MusicBot,:musical_note: The original MusicBot for Disco...,False,https://api.github.com/repos/Just-Some-Bots/Mu...,...,False,False,"[discord, hacktoberfest, musicbot, python]",public,2352,86,2876,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
2,268786553,MDEwOlJlcG9zaXRvcnkyNjg3ODY1NTM=,Discord-MusicBot,SudhanPlayz/Discord-MusicBot,False,"{'login': 'SudhanPlayz', 'id': 55418697, 'node...",https://github.com/SudhanPlayz/Discord-MusicBot,"An advanced discord music bot, supports Spotif...",False,https://api.github.com/repos/SudhanPlayz/Disco...,...,False,False,"[commands, dashboard, discord, discord-bot, di...",public,2089,12,1771,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
3,344891167,MDEwOlJlcG9zaXRvcnkzNDQ4OTExNjc=,SongPlayRoBot,IVETRI/SongPlayRoBot,False,"{'login': 'IVETRI', 'id': 68427115, 'node_id':...",https://github.com/IVETRI/SongPlayRoBot,Add Me To Your Group Enjoy With Me. Pyrogram b...,False,https://api.github.com/repos/IVETRI/SongPlayRoBot,...,False,False,"[musicbot, telegram-bot, telegrambot, telethon...",public,302,15,769,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
4,36205594,MDEwOlJlcG9zaXRvcnkzNjIwNTU5NA==,TS3AudioBot,Splamy/TS3AudioBot,False,"{'login': 'Splamy', 'id': 6502527, 'node_id': ...",https://github.com/Splamy/TS3AudioBot,Advanced Musicbot for Teamspeak 3,False,https://api.github.com/repos/Splamy/TS3AudioBot,...,False,False,"[bot, client, music, playlist, plugin, plugins...",public,127,86,490,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,346900710,MDEwOlJlcG9zaXRvcnkzNDY5MDA3MTA=,MusicBot,JovemHero/MusicBot,False,"{'login': 'JovemHero', 'id': 68248737, 'node_i...",https://github.com/JovemHero/MusicBot,Bot Discord.js de música (funciona na repl.it)!,False,https://api.github.com/repos/JovemHero/MusicBot,...,False,False,[],public,3,2,3,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
96,85118263,MDEwOlJlcG9zaXRvcnk4NTExODI2Mw==,MusicBot,TimovNiedek/MusicBot,False,"{'login': 'TimovNiedek', 'id': 7383203, 'node_...",https://github.com/TimovNiedek/MusicBot,A Telegram chatbot for recommending music base...,False,https://api.github.com/repos/TimovNiedek/MusicBot,...,False,False,[],public,3,0,4,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
97,285854183,MDEwOlJlcG9zaXRvcnkyODU4NTQxODM=,discord-musicbot-citron,yousukeayada/discord-musicbot-citron,False,"{'login': 'yousukeayada', 'id': 62694758, 'nod...",https://github.com/yousukeayada/discord-musicb...,None,False,https://api.github.com/repos/yousukeayada/disc...,...,False,False,[],public,3,0,4,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
98,167591397,MDEwOlJlcG9zaXRvcnkxNjc1OTEzOTc=,ts3j-musicbot,Manevolent/ts3j-musicbot,False,"{'login': 'Manevolent', 'id': 7735455, 'node_i...",https://github.com/Manevolent/ts3j-musicbot,A music bot on top of ts3j,False,https://api.github.com/repos/Manevolent/ts3j-m...,...,False,False,[],public,1,0,4,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0


In [7]:
df2

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,is_template,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,permissions,score
0,94307500,MDEwOlJlcG9zaXRvcnk5NDMwNzUwMA==,Kiu,BjoernPetersen/Kiu,False,"{'login': 'BjoernPetersen', 'id': 2513555, 'no...",https://github.com/BjoernPetersen/Kiu,MusicBot client for Android,False,https://api.github.com/repos/BjoernPetersen/Kiu,...,False,False,[],public,0,19,4,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
1,355408172,MDEwOlJlcG9zaXRvcnkzNTU0MDgxNzI=,Discord-Musicbot,Nich87/Discord-Musicbot,False,"{'login': 'Nich87', 'id': 56439343, 'node_id':...",https://github.com/Nich87/Discord-Musicbot,Simple Discord Musicbot,False,https://api.github.com/repos/Nich87/Discord-Mu...,...,False,False,"[discord, discord-bot, discord-js, music-bot]",public,1,0,4,v13-remaster,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
2,28526881,MDEwOlJlcG9zaXRvcnkyODUyNjg4MQ==,musicbotdosbrodi,Malgnor/musicbotdosbrodi,False,"{'login': 'Malgnor', 'id': 3739867, 'node_id':...",https://github.com/Malgnor/musicbotdosbrodi,None,False,https://api.github.com/repos/Malgnor/musicbotd...,...,False,False,[],public,0,0,4,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
3,142715348,MDEwOlJlcG9zaXRvcnkxNDI3MTUzNDg=,MusicBot,pheonic/MusicBot,False,"{'login': 'pheonic', 'id': 6250817, 'node_id':...",https://github.com/pheonic/MusicBot,A discord bot implementing lavaplayer in Kotlin,False,https://api.github.com/repos/pheonic/MusicBot,...,False,False,[],public,1,0,3,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
4,423152062,R_kgDOGTjJvg,MusicBot,saltukozelgul/MusicBot,False,"{'login': 'saltukozelgul', 'id': 59657199, 'no...",https://github.com/saltukozelgul/MusicBot,a discord bot with button features and music p...,False,https://api.github.com/repos/saltukozelgul/Mus...,...,False,False,[],public,0,0,2,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,398546315,MDEwOlJlcG9zaXRvcnkzOTg1NDYzMTU=,LazyMusicbot,Davidremo02/LazyMusicbot,False,"{'login': 'Davidremo02', 'id': 84329081, 'node...",https://github.com/Davidremo02/LazyMusicbot,Telegram Voice Chat Music,False,https://api.github.com/repos/Davidremo02/LazyM...,...,True,False,[],public,32,0,3,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
96,396714586,MDEwOlJlcG9zaXRvcnkzOTY3MTQ1ODY=,RiZoeLXMusic,MrRizoel/RiZoeLXMusic,False,"{'login': 'MrRizoel', 'id': 85222112, 'node_id...",https://github.com/MrRizoel/RiZoeLXMusic,PRIVATE / PUBLIC MUSICBOT FOR TELEGRAM GROUPS,False,https://api.github.com/repos/MrRizoel/RiZoeLXM...,...,False,False,[],public,17,1,3,Rizoel,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
97,154361901,MDEwOlJlcG9zaXRvcnkxNTQzNjE5MDE=,MusicBot-2,noirscape/MusicBot-2,False,"{'login': 'noirscape', 'id': 13433513, 'node_i...",https://github.com/noirscape/MusicBot-2,Music bot.,False,https://api.github.com/repos/noirscape/MusicBot-2,...,False,False,[],public,6,0,1,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
98,505628440,R_kgDOHiNHGA,MusicBot,bhkvlldu/MusicBot,False,"{'login': 'bhkvlldu', 'id': 104181159, 'node_i...",https://github.com/bhkvlldu/MusicBot,None,False,https://api.github.com/repos/bhkvlldu/MusicBot,...,False,False,[],public,6,0,2,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0


In [8]:
#Get a list of the full_name for the urls
#df.full_name.count()

In [9]:
#df.columns

In [10]:
#df = acquire.scrape_github_data()

In [11]:
#df = pd.DataFrame(df)

In [12]:
#df.to_csv('musicbot_data.csv')

In [13]:
wrangle.get_data()

,repo,language,readme_contents
0,jagrosh/MusicBot,Java,"<img align=""right"" src=""https://i.imgur.com/zr..."
1,Just-Some-Bots/MusicBot,Python,# MusicBot\n\n[![GitHub stars](https://img.shi...
2,SudhanPlayz/Discord-MusicBot,JavaScript,"<h1 align=""center""><img src=""./assets/logo.gif..."
3,IVETRI/SongPlayRoBot,Python,# Check Our New Bot Repo & Video :\n\n[Video](...
4,Splamy/TS3AudioBot,C#,# TS3AudioBot\n\nThis is a open-source TeamSpe...
...,...,...,...
195,Davidremo02/LazyMusicbot,None,"<h2 align=""centre"">Telegram Group Music Player..."
196,MrRizoel/RiZoeLXMusic,Python,"<h2 align=""centre"">ℝ𝚒ℤ𝚘𝚎𝕃𝕏𝕄𝚞𝚜𝚒𝚌 🎵</h2>\n\n### ..."
197,noirscape/MusicBot-2,Python,# Dynamic cog bot template\n\nThis is a bot te...
198,bhkvlldu/MusicBot,Python,# ⌯ Source zoro ˹♻️˼⁩\n\n• I am a developer in...
